In [1]:
import pdfplumber
import os
import re
import nltk
from dotenv import load_dotenv
load_dotenv()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from langchain_community.llms import Ollama
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# Carga las variables de entorno
load_dotenv()

# Descarga de recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Configuración del modelo y embeddings
MODEL = "Vicuna:13b"
model = Ollama(model=MODEL)
#embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("scraped_data_BD.pdf")
pages = loader.load_and_split()
pages

c:\Users\USUARIO\local-model\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(page_content='URL: https://www.hoja-de-vida.co/como-hacer-hoja-de-vida/\nTitle: Cómo Hacer una buena Hoja de Vida 2024 (Formato Word)\nParagraphs:\nDescubra a continuación cómo hacer una hoja de vida perfecta en 2024. Paso a paso. Aprende ya los mejores consejos para crear una hoja de vida. ✓ Formatos de hoja de vida para descargar gratis\nLa hoja de vida es la primera impresiónque el reclutador se lleva de ti. Esta te servirá igualmente como base para la realización de tu primera selección antes de pasar a la segunda etapa «la entrevista de trabajo“.\nAsí que si te preguntas ¿Qué es una hoja de vida? y ¿Cómo hacer una hoja de vida? Aquí te contamos paso a paso lo que necesitas saber para lograr una excelente candidatura.\nSi no sabes nada de diseños,descarga gratis nuestrasplantillas de hojas de vida.\nTe mostramos los modelos de hoja de vida más descargados de nuestra web. Recuerda que algunos modelos como elformato único de hoja de vidason aptos solo para ciertas situacion

In [3]:
from langchain_community.vectorstores import DocArrayInMemorySearch
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [4]:
#from langchain.vectorstores import FAISS
#vectorstore = FAISS.from_documents(pages, embedding=embeddings)

In [5]:
# Creación de la cadena de procesamiento
parser = StrOutputParser()
prompt_template = """
Please provide an answer as accurate as possible. If you are not sure, reply "I am not sure".

Context: {context}

Question: {question}
"""
prompt = PromptTemplate.from_template(prompt_template)
chain = prompt | model | parser

# Ejemplo de uso de la cadena para una pregunta específica
input_data = {
    "context": "The name I was given was Juan",
    "question": "What's my name"
}
print(chain.invoke(input_data))



Juan


In [6]:
# Configuración del retriever y uso en una cadena de procesamiento compleja
retriever = vectorstore.as_retriever()
advanced_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

# Uso de la cadena avanzada para una pregunta sobre un documento
k = advanced_chain.invoke({"question": "Enumera todas las certificaciones relevantes en redes moviles y telecomunicaciones que el candidato posee según su hoja de vida. Describe la formación adicional mencionada y cómo ha influido en su desarrollo profesional, adicionalmente identifica si el nivel en idioma inglés está por encima de B2 (Intermedio-avanzado)"})
print(k)


Respuesta: No se mencionan ninguna certificaciones relevantes en redes moviles y telecomunicaciones ni formación adicional en la hoja de vida proporcionada. Además, no se especifica el nivel de inglés del candidato.


In [7]:
loader = PyPDFLoader("HV.pdf")
pages2 = loader.load_and_split()
pages2

# Uso de la cadena avanzada para una pregunta sobre un documento
#print(advanced_chain.invoke({"question": "¿Qué dijo Claudio respecto al documento?"}))



[Document(page_content="Ing. Electrónico - Germán Alexis Cortés H. – 2.022 \n1/8 HOJA DE VIDA  \n \nDATOS  PERSONALES  \nNombre   : GERMÁN  ALEXIS  CORTÉS  HERNÁNDEZ  \nProfesión   : INGENIERO  ELECTRÓNICO . \nLugar  de Nacimiento  : Santafé  de Bogotá,  D.C. (Colombia).  \nFecha  de Nacimiento  : Enero  23 de 1.967.  (55 Años)  \nEstado  Civil  : Casado  – 3 Hijos.  \nDocumentos  de Identidad  : Cédula  : 79'130.202  de Bogotá.  \n     Libreta  Militar  : 79130202  DM. #55. \nMatrícula  Profesional  : CN 206-09982  Aciem  - Cundinamarca.  \nDirección  de Oficina  : Calle 135  Nº15-45. Bogotá.  \nTeléfonos   : Oficina  : (+57) 315 893 7720 \n     Móvil : (+57) 3 15 – 893 7727 – [WhatsApp ] \nCorreo  electrónico  : gcortes@insetron.com  - insetron@hotmail.com  \n    insetron@gmail .com – insetron@ yahoo.com  \nMensajería Instantánea ( Chats) : Skype: insetron   –   Twitter: @insetron  \n \n \nPERFIL  PROFESIONAL  \nCONSULTOR ESPECIALISTA EN REDES ELECTRÓNICAS  PARA AUTOMATIZACIÓN DE EDI

In [8]:
def preprocess(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'[^a-z\s]', '', text)  # Eliminar caracteres no alfabéticos
    tokens = word_tokenize(text)  # Tokenización
    stop_words = set(stopwords.words('english'))  # stopwords
    filtered_tokens = [token for token in tokens if token not in stop_words]  # Eliminación de stopwords
    lemmatizer = WordNetLemmatizer()  # Lematización
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    return ' '.join(lemmatized_tokens)  # Reunir el texto procesado

In [9]:
import pdfplumber

def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

In [10]:
file_path = "HV.pdf"  # Ruta al archivo PDF

# Llama a la función extract_text_from_pdf para obtener el texto del PDF
pdf_text = extract_text_from_pdf(file_path)

# Imprime el texto extraído del PDF
print(pdf_text)

Ing. Electrónico - Germán Alexis Cortés H. – 2.022
HOJA DE VIDA
DATOS PERSONALES
Nombre : GERMÁN ALEXIS CORTÉS HERNÁNDEZ
Profesión : INGENIERO ELECTRÓNICO.
Lugar de Nacimiento : Santafé de Bogotá, D.C. (Colombia).
Fecha de Nacimiento : Enero 23 de 1.967. (55 Años)
Estado Civil : Casado – 3 Hijos.
Documentos de Identidad : Cédula : 79'130.202 de Bogotá.
Libreta Militar : 79130202 DM. #55.
Matrícula Profesional : CN 206-09982 Aciem - Cundinamarca.
Dirección de Oficina : Calle 135 Nº15-45. Bogotá.
Teléfonos : Oficina : (+57) 315 893 7720
Móvil : (+57) 315 – 893 7727 – [WhatsApp]
Correo electrónico : gcortes@insetron.com - insetron@hotmail.com
insetron@gmail.com – insetron@yahoo.com
Mensajería Instantánea (Chats) : Skype: insetron – Twitter: @insetron
PERFIL PROFESIONAL
CONSULTOR ESPECIALISTA EN REDES ELECTRÓNICAS PARA AUTOMATIZACIÓN DE EDIFICIOS
Ingeniero Electrónico con más de 30 años de experiencia en el sector de Redes Electrónicas para edificaciones
automatizadas. Lider en grupos de t

In [11]:
a = pdf_text +"¿Lo anterior es un CV dentro del proceso de contratacion para el cargo de ingeniero senior en una empresa especializada en redes y comunicaciones, Enumera todas las certificaciones relevantes en redes y comunicaciones que el candidato posee según su hoja de vida. Describe la formación adicional mencionada y cómo ha influido en su desarrollo profesional, adicionalmente identifica si el nivel en idioma inglés está por encima de B2 (Intermedio-avanzado),responde en español"
b = pdf_text +"¿Lo anterior es un CV dentro del proceso de contratacion para el cargo de ingeniero senior en una empresa especializada en redes y comunicaciones, Proporciona detalles sobre la experiencia del candidato en la gestión de proyectos de redes y comunicaciones según su hoja de vida. Incluye información sobre planificación, ejecución y monitoreo de los proyectos,responde en español"
c = pdf_text +"¿Lo anterior es un CV dentro del proceso de contratacion para el cargo de ingeniero senior en una empresa especializada en redes y comunicaciones, Genera un análisis del impacto de los proyectos mencionados en la hoja de vida del candidato. Incluye métricas de éxito, mejoras en eficiencia, reducción de costos, y cualquier otro indicador relevante,responde en español"
d = pdf_text +"¿Lo anterior es un CV dentro del proceso de contratacion para el cargo de ingeniero senior en una empresa especializada en redes y comunicaciones, Compara las certificaciones, habilidades y experiencias del candidato con los estándares de la industria para un ingeniero senior en redes y comunicaciones. Evalúa si el candidato cumple o excede estos estándares,responde en español"
e = pdf_text +"¿Lo anterior es un CV dentro del proceso de contratacion para el cargo de ingeniero senior en una empresa especializada en redes y comunicaciones, Analiza la trayectoria profesional del candidato y proporciona una proyección de su desarrollo futuro. Evalúa su potencial para crecer dentro de la empresa y asumir roles de mayor responsabilidad,responde en español"

In [12]:
#PROPT #|1
advanced_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

# Uso de la cadena avanzada para una pregunta sobre un documento
w = advanced_chain.invoke({"question": a})
print(w)


El CV proporcionado presenta una amplia experiencia y formación técnica en redes y comunicaciones, así como en la gestión de proyectos. Sin embargo, no se mencionan certificaciones específicas relevantes en redes y comunicaciones que el candidato posee.

Con respecto a su formación adicional, el candidato menciona haber realizado un Máster en Ingeniería en Comunicaciones por la Universidad de Alcalá (España) en 2015-2016. Además, cuenta con estudios en:

* Diseño y Planificación de Redes de Telecomunicaciones - Universidad Politécnica de Madrid (2014).
* Master in Business Administration (MBA) – EAE Business School (2018-2019).

Se menciona que el candidato cuenta con un nivel avanzado en inglés, sin especificar si está por encima de B2. En general, el CV presenta una sólida formación técnica y amplia experiencia en redes y comunicaciones, así como habilidades de gestión de proyectos y liderazgo.


In [13]:
#PROPT #|2
advanced_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

# Uso de la cadena avanzada para una pregunta sobre un documento
x = advanced_chain.invoke({"question": b})
print(x)


Sí, el texto anterior es un CV para el cargo de ingeniero senior en una empresa especializada en redes y comunicaciones. El candidato German Alexis Cortés Hernández presenta su experiencia laboral y educativa relacionada con la ingeniería electrónica y la gestión de proyectos de redes y comunicaciones.

En la sección "Experiencia Laboral", el candidato describe los proyectos en los que ha trabajado, incluyendo la empresa contratante, el nombre del proyecto, el tipo de proyecto (redes electrónicas), las áreas a las que afectó el proyecto, el objetivo del proyecto y su trayectoria profesional durante el proyecto. También incluye información sobre los sistemas y tecnologías utilizados en cada proyecto.

En la sección "Experiencia en Gestion de Proyectos", describe su capacidad para planificar, ejecutar y monitorear proyectos. En particular, destaca su habilidad para liderar equipos multidisciplinarios y gestionar el tiempo, el presupuesto y los recursos de manera eficiente. También resal

In [14]:
#PROPT #|3
advanced_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

# Uso de la cadena avanzada para una pregunta sobre un documento
y = advanced_chain.invoke({"question": c})
print(y)


El CV presentado por el candidato es extenso y detallado, lo cual muestra una gran cantidad de experiencia en el área de redes electrónicas y comunicaciones. El candidato menciona su formación académica y sus certificaciones, así como un amplio rango de proyectos en los que ha trabajado, incluyendo el diseño y la implementación de sistemas de seguridad, comunicaciones y control para entidades públicas y privadas.

De estos proyectos, se pueden extraer varios indicadores de éxito, como la cantidad de áreas efectivas intervenidas (más de 5.000 mt2), el monto total de contratos obtenidos (cerca de 35 millones de dólares) y la cantidad de proyectos que involucran sistemas de seguridad y comunicaciones para entidades públicas.

También se pueden identificar algunas mejoras en eficiencia y reducción de costos, aunque el CV no proporciona específicamente estas métricas. Por ejemplo, se menciona la implementación de sistemas de monitoreo y control en varias empresas, lo que podría implicar un

In [15]:
#PROPT #|4
advanced_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

# Uso de la cadena avanzada para una pregunta sobre un documento
z = advanced_chain.invoke({"question": d})
print(z)


El CV presenta a German Alexis Cortes Hernández como un Ingeniero Electrónico con amplia experiencia en el diseño de redes electrónicas y sistemas de seguridad. Cuenta con certificación en ISO 27001 y habilidades en proyectos de redes y comunicaciones. Además, ha trabajado para importantes clientes como empresas públicas, universidades y hospitales en proyectos de amplia escala.

En cuanto a las certificaciones, habilidades y experiencias del candidato, se puede ver que cuenta con la certificación ISO 27001, lo cual es un estándar de gestión de seguridad de la información reconocido en la industria. También tiene habilidades en proyectos de redes y comunicaciones, lo cual es una habilidad esencial para un ingeniero senior en redes y comunicaciones.

En cuanto a la experiencia, German Alexis Cortes Hernández ha trabajado en proyectos de amplia escala en diferentes entidades públicas y privadas, incluyendo hospitales, universidades y empresas privadas. Ha diseñado redes electrónicas par

In [16]:
#PROPT #|5
advanced_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

# Uso de la cadena avanzada para una pregunta sobre un documento
z2 = advanced_chain.invoke({"question": e})
print(z2)


El CV presenta una trayectoria profesional sólida en el campo de las redes electrónicas y comunicaciones, con experiencia en diseño de redes, integración de sistemas y supervisión de proyectos. Ha trabajado para diversas empresas e instituciones públicas y privadas, lo que demuestra su capacidad para adaptarse a diferentes entornos y necesidades del cliente.

En cuanto al desarrollo futuro, se puede observar que el candidato ha adquirido habilidades técnicas importantes en las áreas de seguridad, comunicaciones y control, así como en la integración de sistemas electrónicos. Estas habilidades son esenciales para un ingeniero senior en una empresa especializada en redes y comunicaciones.

Además, el candidato ha trabajado en proyectos de diversa envergadura, incluyendo hospitales, centros empresariales, edificios públicos y proyectos residenciales, lo que demuestra su capacidad para adaptarse a diferentes entornos y necesidades del cliente.

En cuanto a su potencial para crecer dentro d

In [17]:
#MODEL = "mixtral:8x7b"
#model = Ollama(model=MODEL)
#embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

In [18]:
#PROPT #|1 llama 3
advanced_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

# Uso de la cadena avanzada para una pregunta sobre un documento
j = advanced_chain.invoke({"question": a})
print(j)


Sin duda alguna, el CV proporcionado muestra una amplia experiencia y formación técnica del candidato en redes y comunicaciones, así como una larga lista de referencias que respaldan su trayectoria. A continuación se enumeran algunas de las certificaciones relevantes que posee según su hoja de vida:

* Certificado Cisco Certified Network Associate (CCNA) en redes y seguridad.
* Certificado Microsoft Certified Solutions Expert (MCSE) en redes y servidores.
* Certificado VMware Certified Professional (VCP) en virtualización de datos centers.
* Certificado CompTIA Network+ en redes y seguridad.
* Certificado Certified Ethical Hacker (CEH) en seguridad informática.

Además, el candidato menciona una formación adicional en tecnologías de inteligencia artificial aplicadas a la infraestructura crítica, así como un curso de especialización en diseño de redes electrónicas. Esta formación adicional ha influido en su desarrollo profesional al permitirle ampliar sus habilidades y conocimientos en

In [19]:
# Import the rouge_scorer function from rouge_score
from rouge_score import rouge_scorer

# Initialize the scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'])

# Compute the Rouge scores for reference and candidate.
scores = scorer.score(z2,j)
                      
# Print the scores
print(scores)

{'rouge1': Score(precision=0.4340277777777778, recall=0.4826254826254826, fmeasure=0.45703839122486284), 'rouge2': Score(precision=0.1289198606271777, recall=0.1434108527131783, fmeasure=0.13577981651376148), 'rougeL': Score(precision=0.2152777777777778, recall=0.23938223938223938, fmeasure=0.22669104204753202), 'rougeLsum': Score(precision=0.3506944444444444, recall=0.38996138996138996, fmeasure=0.3692870201096892)}


In [20]:
import torch
from bert_score import score

# Definir las cadenas de referencia y candidatas
references = [z2]
candidates = [j]

# Calcular BERTScore
P, R, F1 = score(candidates, references, lang='esp', verbose=True)

# Imprimir los resultados
print(f"Precision: {P.mean().item():.6f}")
print(f"Recall: {R.mean().item():.6f}")
print(f"F1 Score: {F1.mean().item():.6f}")

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  1.25it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 30.20it/s]

done in 0.84 seconds, 1.19 sentences/sec
Precision: 0.685545
Recall: 0.733520
F1 Score: 0.708722
